In [8]:
import json
import pennylane as qml
import pennylane.numpy as np
import itertools


# Put any helper functions here that you want to make #
#####################################
def qft(q, n):
    # Put the gates of the QFT circuit
    for j in range(n):
        
        qml.Hadamard(wires=q[j])
        for k in range(j+1,n):
            qml.ControlledQubitUnitary(qml.U1(np.pi/float(2**(k-j)), wires=q[j]), control_wires=q[k], wires=q[j])
    
    # Swap qubits
    for j in range(n//2):
        qml.SWAP(wires=[q[j], q[n-j-1]])
    return qft
        
def inv_qft(q, n):
    # Swap qubits
    for j in range(n//2):
        qml.SWAP(wires=[q[j], q[n-j-1]])
    
    # Put the gates of the inverse QFT circuit
    for j in reversed(range(n)):
        for k in reversed(range(j+1,n)):
            qml.ControlledQubitUnitary(qml.U1(-np.pi/float(2**(k-j)), wires=q[j]), control_wires=q[k], wires=q[j])
        qml.Hadamard(wires=q[j])
    return inv_qft

def n_cry(theta, control_qubits, target_qubit, inv = False):
    
    # Convert specified control qubits to a list
    control_list = control_qubits[:]
    size = len(control_list)
    
    
    if size > 10:
        print('Argument n is invalid!')
    else:
            
        if size == 1:
            qml.CRY(theta, wires=(control_qubits[0], target_qubit))
        else:
            # To understand this part, please check section 6.3.1 of the report
            new_controls = control_qubits[:-1] if inv else control_qubits[1:]
            qml.MultiControlledX(control_wires=control_qubits, wires=target_qubit)
            
            n_cry(-theta/2, new_controls, target_qubit, inv)
            qml.MultiControlledX(control_wires=control_qubits, wires=target_qubit)
            n_cry(theta/2, new_controls, target_qubit, inv)
    return n_cry
        
def postselect(statevector, qubit_number, value: bool):
    
    # Define a mask depending on the specified qubit
    mask = 1 << qubit_number
    
    # Depending on the desired value of the qubit, update the quantum state
    if value:
        array_mask = np.arange(len(statevector)) & mask
        array_mask = (array_mask != 0)
    else:
        array_mask = np.logical_not(np.arange(len(statevector)) & mask)
        
    def normalize(v):
        norm = np.linalg.norm(v)
        if norm == 0: 
            return v
        return v / norm
    
    # Return a normalized quantum state
    return normalize(statevector[array_mask])


def qpe(clock_reg, target):
    
    # Perform a Hadamard Transform
    for q in clock_reg:
        qml.Hadamard(wires=q)
    
    # Apply successive powers of our operator
    qml.Hadamard(wires=target)
    
    qml.CRZ(3*np.pi/2, wires=[clock_reg[2], target[0]])
    qml.Hadamard(wires=target)
    
    qml.Hadamard(wires=target)
    qml.CRZ(-np.pi, wires=[clock_reg[1], target[0]])
    qml.Hadamard(wires=target)
    
    qml.CRZ(2*np.pi, wires=[clock_reg[0], target[0]])
    
    # Perform an inverse QFT on the register holding the eigenvalues
    inv_qft(clock_reg, 3)
    return qpe
    
def inv_qpe(clock_reg, target):
    
    # Perform a QFT on the register holding the eigenvalues
    qft(clock_reg, 3)
    
    # Apply successive powers of our operator (in reverse)
    qml.CRZ(-2*np.pi, wires=[clock_reg[0], target[0]])
    
    qml.Hadamard(wires=target)
    qml.CRZ(np.pi, wires=[clock_reg[1], target[0]])
    qml.Hadamard(wires=target)
    
    qml.Hadamard(target)
    qml.CRZ(-3*np.pi/2, wires=[clock_reg[2], target[0]])
    qml.Hadamard(wires=target)
    
    # Perform a Hadamard Transform
    for q in clock_reg:
        qml.Hadamard(q)
        
    return inv_qpe

def multi_controlled_RY(angle, control_wires, target_wire):
    if len(control_wires) == 1:
        qml.CRY(angle, wires=control_wires + [target_wire])
    elif len(control_wires) > 1:
        qml.Hadamard(wires=target_wire)
        qml.MultiControlledX(control_wires=control_wires, wires=target_wire)
        qml.RZ(angle, wires=target_wire)
        qml.MultiControlledX(control_wires=control_wires, wires=target_wire)
        qml.Hadamard(wires=target_wire)
    return multi_controlled_RY



#qml.ControlledQubitUnitary(encode_hermitian(A, b_wires), control_wires=qpe_wires, wires=b_wires)
#######################################################################
def encode_hermitian(A, wires):
    """Encodes a hermitian matrix A as a unitary U = e^{iA}

    Args
        A (numpy.tensor): a 2x2 matrix
        b (numpy.tensor): a length-2 vector

    Returns
        (qml.Operation): a unitary operation U = e^{iA}
    """
    return qml.exp(qml.Hermitian(A, wires=wires), coeff=1j)


def mint_to_lime(A, b):
    """Calculates the optimal mint and lime proportions in the Mojito HHLime twist.

    Args
        A (numpy.tensor): a 2x2 matrix
        b (numpy.tensor): a length-2 vector

    Returns
        x (numpy.tensor): the solution to Ax = b
        (int): the number of operations in your HHL circuit.
    """
    b_qubits = 1
    b_wires = [0]

    qpe_qubits = 10
    qpe_wires = list(range(b_qubits, b_qubits + qpe_qubits))

    ancilla_qubits = 1
    ancilla_wires = list(
        range(b_qubits + qpe_qubits, ancilla_qubits + b_qubits + qpe_qubits)
    )

    all_wires = b_wires + qpe_wires + ancilla_wires
    dev = qml.device("default.qubit", wires=all_wires)

    @qml.qnode(dev)
    def HHL(A, b):
        """Implements the HHL algorithm.
        Args
            A (numpy.tensor): a 2x2 matrix
            b (numpy.tensor): a length-2 vector

        Returns
            x^2 (numpy.tensor):
                The probability distribution for the vector x, which is the
                solution to Ax = b.
        """


        # Put your code here #
        
        # Perform the QPE designed specifically for our problem

        for i in range(1, 2**10):
            bin_str = format(i, '010b')
            
            control_qubits = list([int(index)+1 for index, bit in enumerate(bin_str) if bit == '1'])
            # print(control_qubits)
            
            decimal_val = 1/i
            theta = 2 * np.arcsin(decimal_val)
            theta = theta/len(control_qubits)
            multi_controlled_RY(theta, control_qubits, ancilla_wires[0])
            
            # qml.MultiControlledX(control_wires=control_qubits, wires=ancilla_qubits)
            # qml.CRY(theta, wires=(control_qubits, [0]))
            # qml.MultiControlledX(control_wires=control_qubits, wires=ancilla_qubits)
            
            # k = i
            # for j in range(10):
            #     remainder = k%2
            #     k = k/2
            #     if (remainder==1):
            #         qml.CRY(theta, wires=[qpe_wires[j], ancilla_wires[0]])
                    
                    
        # qml.CRY(2.89385, wires=[qpe_wires[1], ancilla_wires[0]])
        # qml.CRY(1.69612, wires=[qpe_wires[0], ancilla_wires[0]])
        # n_cry(0.24774, qpe_wires[1:], ancilla_wires[0], inv=True)
        # n_cry(-0.40912, qpe_wires[::2], ancilla_wires[0], inv=True)
        # n_cry(-3.54277, qpe_wires[:2], ancilla_wires[0], inv=True)
        
        # Perform the inverse QPE specifically designed for our problem
        inv_qpe(qpe_wires, b_wires)
        
        return qml.probs(wires=b_wires)

    # we return probs, but we need the state itself (it will be real-valued)
    x = np.sqrt(HHL(A, b))
    
    
    return x, len(HHL.tape._ops)


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    A, b = json.loads(test_case_input)
    output, num_ops = mint_to_lime(np.array(A), np.array(b))
    output = output.tolist()
    output.append(num_ops)
    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    output = solution_output[:-1]
    num_ops = solution_output[-1]
    expected_output = json.loads(expected_output)
    print(output)

    assert num_ops > 4, "Your circuit should have a few more operations!"
    #assert np.allclose(output, expected_output, rtol=1e-2)


# These are the public test cases
test_cases = [
    ('[[[1, -0.333333], [-0.333333, 1]], [0.48063554, 0.87692045]]', '[0.6123100731658992, 0.7906177169127275]'),
    ('[[[0.456, -0.123], [-0.123, 0.123]], [0.96549299, 0.26042903]]', '[0.5090526763759141, 0.8607353673888718]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[[1, -0.333333], [-0.333333, 1]], [0.48063554, 0.87692045]]'...
[0.7071067811864362, 0.7071067811864362]
Correct!
Running test case 1 with input '[[[0.456, -0.123], [-0.123, 0.123]], [0.96549299, 0.26042903]]'...
[0.7071067811864362, 0.7071067811864362]
Correct!
